<a href="https://colab.research.google.com/github/nssn96/ML_imageclassifier/blob/main/ML_imageClassifier.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# ML Assignment 5 - Image Classification


1.   Author : Surya Narayanan Nadhamuni Suresh
2.   UTA ID : 1001877873


In [9]:
#import lines
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

import torch
import torchvision
import torchvision.transforms as transforms
import tensorflow_datasets as tfds
from torchvision.datasets import Food101

In [4]:
#References
#https://www.geeksforgeeks.org/training-neural-networks-using-pytorch-lightning/
#https://www.kaggle.com/code/uzairshahmdn/food101-midterm-exam
#https://www.kaggle.com/code/darthgera/food101-resnet-50-pytorch
#https://www.kaggle.com/code/ajaykrishnabandari/food-101
#https://www.kaggle.com/code/odednir/multiclass-food-classification-using-tensorflow

In [ ]:
#Installing pytorch lighting
#pip install pytorch-lightning lightning-bolts
!pip install pytorch-lightning


In [10]:
import pytorch_lightning as pl

In [ ]:
#Reference Used for loading the dataset
#https://colab.research.google.com/drive/1Z5-LqlJxNslHcCzM9jChsNhGi9Y7HSC4#scrollTo=f88XvEBTQBS9
#Loading the food101 dataset
#food, info = tfds.load("food101", with_info=True)
#ds = tfds.load('huggingface:food101')
import os
def get_data_extract():
  if "food-101" in os.listdir():
    print("Dataset already exists")
  else:
    print("Downloading the data...")
    !wget http://data.vision.ee.ethz.ch/cvl/food-101.tar.gz
    print("Dataset downloaded!")
    print("Extracting data..")
    !tar xzvf food-101.tar.gz
    print("Extraction done!")

# get_data_extract()

In [18]:
#contents inside the folder
!ls food-101/

images	license_agreement.txt  meta  README.txt


In [21]:
!ls food-101/meta

classes.txt  labels.txt  test.json  test.txt  train.json  train.txt


From above output we can see that we have the train dataset in train.txt and the test dataset in the test.txt

In [24]:
#checking some of the data in the train.txt file
!head 'food-101/meta/train.txt'

apple_pie/1005649
apple_pie/1014775
apple_pie/1026328
apple_pie/1028787
apple_pie/1043283
apple_pie/1050519
apple_pie/1057749
apple_pie/1057810
apple_pie/1072416
apple_pie/1074856


image ids are unique and correspond to the foodspotting.com review ids

In [ ]:
#The image folder contains 101 folders of food class with 1000 images in each
os.listdir('food-101/images')